In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import pathlib


In [31]:
# load a test data set got one pattern

sample_95 = gpd.read_parquet("../cta-stop-etl/out/trips/test_trips_95_small.parquet")

sample_3910 = gpd.read_parquet("../cta-stop-etl/out/trips/test_trips_3910_small.parquet")

In [32]:
sample_95['pid'] = '95'
sample_3910['pid'] = '3910'

In [ ]:
# can add in bus locations at some point and look at those
#sample['time'] = sample[['bus_stop_time', 'data_time']].bfill(axis=1).iloc[:, 0]
#sample.drop(columns=['bus_stop_time', 'data_time'], inplace=True)

In [35]:
sample_95_stops = sample_95[sample_95['typ']== 'S']
sample_3910_stops = sample_3910[sample_3910['typ']== 'S']
sample = pd.concat([sample_95_stops,sample_3910_stops])
sample= sample[['unique_trip_vehicle_day','pid','seg_combined', 'stpid', 'p_stp_id', 'geometry', 'bus_stop_time']]
sample.head()


,unique_trip_vehicle_day,pid,seg_combined,stpid,p_stp_id,geometry,bus_stop_time
0,7295.0235314066107561419402023-01-04,95,0.0,923,95-923,POINT (358249.87291 582489.58505),2023-01-04 23:36:45.000000000
7,7295.0235314066107561419402023-01-04,95,4.0,924,95-924,POINT (358177.75661 582481.33075),2023-01-04 23:47:32.408154452
8,7295.0235314066107561419402023-01-04,95,9.0,926,95-926,POINT (357979.93959 582483.38174),2023-01-04 23:47:55.400228857
9,7295.0235314066107561419402023-01-04,95,16.0,927,95-927,POINT (357690.92237 582477.25608),2023-01-04 23:48:55.847421941
10,7295.0235314066107561419402023-01-04,95,21.0,928,95-928,POINT (357449.61839 582470.41645),2023-01-04 23:49:46.136671140


In [34]:
# 100 trips of each pattern 

sample.groupby('pid')['unique_trip_vehicle_day'].nunique()

pid
3910    100
95      100
Name: unique_trip_vehicle_day, dtype: int64

by bus stop pattern id
how many buses stop per day


total number of days that we have trips for
per pattern
    how many pings per hours
    number of pings for that bus stop / total days we had trips for that pattern

In [37]:
sample['bus_stop_time'].dt.date

0       2023-01-04
7       2023-01-04
8       2023-01-04
9       2023-01-04
10      2023-01-04
           ...    
8260    2023-02-16
8261    2023-02-16
8262    2023-02-16
8263    2023-02-16
8265    2023-02-16
Name: bus_stop_time, Length: 14300, dtype: object

In [ ]:
group by the pid 

In [43]:
test = sample[sample['pid']== '95']
total_trip_days = test['bus_stop_time'].dt.date.nunique()
test.groupby('p_stp_id').size()



p_stp_id
95-1000    100
95-1001    100
95-1002    100
95-1004    100
95-1005    100
          ... 
95-993     100
95-996     100
95-997     100
95-998     100
95-999     100
Length: 73, dtype: int64

In [22]:
for pid in sample['pid'].unique():
    print(pid)
    total_trip_days = sample['bus_stop_time'].dt.date.nunique()
    samp

sample['hour'] = sample['time'].dt.hour
sample['day_of_year'] = sample['time'].dt.day_of_year

In [23]:
print(sample['time'].min())
print(sample['time'].max())

2023-01-03 06:32:00
2023-01-07 00:52:00


In [29]:
test_group = sample.groupby(['p_stp_id', 'day_of_year','hour']).size().reset_index(name='count')
test_group[test_group["p_stp_id"] == '95-999']

test_group.groupby(['p_stp_id', 'hour']).mean().reset_index()

,p_stp_id,hour,day_of_year,count
0,14111-10561,6,4.5,1.00
1,14111-10561,7,4.5,3.50
2,14111-10561,8,4.5,3.50
3,14111-10561,9,4.5,3.75
4,14111-10561,10,4.5,4.00
...,...,...,...,...
945,95-999,19,4.5,3.25
946,95-999,20,4.5,3.50
947,95-999,21,4.5,1.75
948,95-999,22,4.5,2.25
